In [29]:
# Loop through the directory; find data from each json; look in the text directories; open each subdirectory;
# read in each subdirectory json file; find the one with the latest date. Read and store text file associated
# with that directory

In [30]:
# Set up the database to save the results of the new york bill table
# There will be one table for the New York bills and one for U.S. bills
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

In [31]:
import os

In [32]:
os.getcwd()

'/Users/joeljoel/Documents/projects/identifyabill/bill_taxonomy/notebooks'

In [33]:
#In Python: Define a database name
dbname = 'bills_db'
username = 'joeljoel'
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgresql://%s@localhost/%s'%(username,dbname))
print(engine.url)

## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

postgresql://joeljoel@localhost/bills_db
True


In [34]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String
class US_Bill(Base):
    __tablename__ = 'us_bills'
    bill_num = Column(String, primary_key=True)
    bill_name = Column(String)
    bill_text = Column(String)
    top_subject = Column(String)

    def __repr__(self):
        return "<US_Bill(bill_num='%s', bill_name='%s', bill_text='%s,', top_subject='%s,')>" % (
            self.bill_num, self.bill_name, self.bill_text, self.top_subject)

/var/folders/qb/nxfghts14xv2gh9svxdkp3d40000gn/T/ipykernel_1187/1992495011.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [35]:
# Actually create the table
Base.metadata.create_all(engine)

In [36]:
class Bill_Subject(Base):
    __tablename__ = 'bill_subject'
    bill_num = Column(String, primary_key=True)
    subject = Column(String, primary_key=True)

    def __repr__(self):
        return "<Bill_Subject(bill_num='%s', subject='%s')>" % (
            self.bill_num, self.subject)

In [37]:
#session.close()
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [38]:
import json
import os
import datetime
def get_latest_text_dir(path):
    latest_date = datetime.datetime(1900, 1, 1)
    dir_name = ""
    for status_dir in os.listdir(path):  
        status_file = path + '/' + status_dir + '/data.json'
        try:
            with open(status_file) as data_file:
                status_data = json.load(data_file)
                date = datetime.datetime.strptime(status_data['issued_on'], '%Y-%m-%d')
                if (date > latest_date):
                    latest_date = date
                    dir_name = status_dir
        except:
            print(f"Failed on: {status_file}!")
            continue
    # print(latest_date)
    return dir_name

In [39]:
# print(get_latest_text_dir('/Users/Joel/Desktop/Insight/data/bills/s/s2039/text-versions'))

In [40]:
Bill_Subject.__table__

Table('bill_subject', MetaData(), Column('bill_num', String(), table=<bill_subject>, primary_key=True, nullable=False), Column('subject', String(), table=<bill_subject>, primary_key=True, nullable=False), schema=None)

In [42]:
import os
import json
from sqlalchemy.exc import IntegrityError, SQLAlchemyError

# Assuming US_Bill and Bill_Subject are already defined
# Assuming session is already defined

bills_dir = '/Users/joeljoel/Google Drive/My Drive/Insight/bill_taxonomy/data/raw/bills'

for bill_type_dir in os.listdir(bills_dir):
    type_dir = os.path.join(bills_dir, bill_type_dir)
    for i, bill_dir in enumerate(os.listdir(type_dir)):
        bill_path = os.path.join(type_dir, bill_dir)
        text_dir_base = os.path.join(bill_path, 'text-versions')
        if os.path.isdir(text_dir_base):
            dir_name = get_latest_text_dir(text_dir_base)
            text_name = os.path.join(text_dir_base, dir_name, 'document.txt')
            
            text = ""
            try:
                with open(text_name) as text_file:
                    text = text_file.readlines()
            except FileNotFoundError:
                print(f"Missing bill: {text_name}!")
                continue
            
            outer_json = os.path.join(bill_path, 'data.json')
            with open(outer_json) as data_file:    
                bill_data = json.load(data_file)
                bill_num = bill_data['bill_id']
                bill_name = bill_data['official_title']
                top_subject = bill_data['subjects_top_term']
                
                one_bill = US_Bill(bill_num=bill_num, bill_name=bill_name, bill_text=text, top_subject=top_subject)
                
                try:
                    session.add(one_bill)
                    session.commit()
                except IntegrityError:
                    print(f"Duplicate bill entry skipped: {bill_num}")
                    session.rollback()  # Rollback the transaction on error
                except SQLAlchemyError as e:
                    print(f"An error occurred: {e}")
                    session.rollback()

                for term in bill_data['subjects']:
                    exists = session.query(Bill_Subject).filter_by(bill_num=bill_num, subject=term).first()
                    if not exists:
                        one_sub = Bill_Subject(bill_num=bill_num, subject=term)
                        try:
                            session.add(one_sub)
                            session.commit()
                        except IntegrityError:
                            print(f"Duplicate subject entry skipped for bill {bill_num}: {term}")
                            session.rollback()  # Rollback the transaction on error
                        except SQLAlchemyError as e:
                            print(f"An error occurred: {e}")
                            session.rollback()

                if i % 100 == 0:
                    print(i, bill_data['bill_id'])


Duplicate bill entry skipped: hjres58-114
0 hjres58-114
Duplicate bill entry skipped: hjres93-114
Duplicate bill entry skipped: hjres67-114
Duplicate bill entry skipped: hjres60-114
Duplicate bill entry skipped: hjres94-114
Duplicate bill entry skipped: hjres69-114
Duplicate bill entry skipped: hjres56-114
Duplicate bill entry skipped: hjres51-114
Duplicate bill entry skipped: hjres34-114
Duplicate bill entry skipped: hjres33-114
Duplicate bill entry skipped: hjres50-114
Duplicate bill entry skipped: hjres68-114
Duplicate bill entry skipped: hjres57-114
Duplicate bill entry skipped: hjres95-114
Duplicate bill entry skipped: hjres61-114
Duplicate bill entry skipped: hjres59-114
Duplicate bill entry skipped: hjres66-114
Duplicate bill entry skipped: hjres92-114
Duplicate bill entry skipped: hjres32-114
Duplicate bill entry skipped: hjres35-114
Duplicate bill entry skipped: hjres19-114
Duplicate bill entry skipped: hjres26-114
Duplicate bill entry skipped: hjres21-114
Duplicate bill entry